In [ ]:
#| default_exp run

In [ ]:
#| hide
from nbdev.showdoc import *

In [ ]:
#| export 

import torch
from torch import nn
import torch.nn.functional as F
import pytorch_lightning as pl
from pytorch_lightning.callbacks import TQDMProgressBar
from torchmetrics import AUROC
from sklearn.metrics import roc_auc_score

from typing import Any, Union, Tuple

In [ ]:
#| export
_size_2_t = Union[int, Tuple[int, int]]

In [ ]:
sample_data = torch.rand((2, 1, 32, 32))
sample_data.shape

torch.Size([2, 1, 32, 32])

In [ ]:
labels = torch.rand((10, 1))
acts = torch.rand((10, 10))
acts.shape, labels.shape

(torch.Size([10, 10]), torch.Size([10, 1]))

In [ ]:
nn.CrossEntropyLoss()(acts, labels.to(torch.long).squeeze())

tensor(2.4210)

In [ ]:
preds = torch.tensor([0.13, 0.26, 0.08, 0.19, 0.34])
target = torch.tensor([0, 0, 1, 1, 1])
auroc = AUROC(task="binary")
auroc(preds, target)

tensor(0.5000)

In [ ]:
# | export


class CIFARLightningModule(pl.LightningModule):
    def __init__(
        self,
        model,
        dls,
        lr=3e-3,
        wd=1e-4,
        loss_func=nn.CrossEntropyLoss,
        verbose=True,
        patience=10,
        num_classes=10,
    ):
        super(CIFARLightningModule, self).__init__()
        self.model = model(n_cls=num_classes)
        self.verbose = verbose
        self.patience = patience
        self.loss_func = loss_func(label_smoothing=0.1)
        self.train_metric = AUROC(task="multiclass", num_classes=num_classes)
        self.val_metric = AUROC(task="multiclass", num_classes=num_classes)
        self.lr = lr
        self.wd = wd
        self.train_dls, self.valid_dls = dls
        self.train_step_outputs = []
        self.val_step_outputs = []
        self.save_hyperparameters(ignore="model")

    def forward(self, x):
        return self.model(x)

    def train_dataloader(self):
        return self.train_dls

    def val_dataloader(self):
        return self.valid_dls

    def training_step(self, batch, bidx):
        images, labels = batch
        logits = self(images)
        acts = F.softmax(logits, dim=-1)
        preds = acts.argmax(-1).squeeze()
        pred_proba = acts.detach()
        labels = labels.to(torch.long)
        train_loss = self.loss_func(logits.squeeze(), labels.squeeze())
        score = self.train_metric(pred_proba, labels.squeeze())
        self.log(
            "train_score_step",
            score,
            on_step=True,
            on_epoch=False,
            prog_bar=False,
            add_dataloader_idx=True,
            logger=True,
        )
        self.log(
            "train_loss_step",
            train_loss,
            on_step=True,
            on_epoch=False,
            prog_bar=False,
            add_dataloader_idx=True,
            logger=True,
        )
        train_logs = {
            # "loss": train_loss,  # requires 'loss' key
            "train_loss": train_loss,
            "train_preds_step": preds,
            "train_score_step": score,
            "train_pred_proba_step": pred_proba,
            "train_labels_step": labels.squeeze().cpu(),
        }
        self.train_step_outputs.append(train_logs)
        return train_loss

    def validation_step(self, batch, bidx):
        images, labels = batch
        logits = self(images)
        acts = F.softmax(logits, dim=-1)
        preds = acts.argmax(-1).squeeze()
        pred_proba = acts.detach()
        labels = labels.to(torch.long)
        valid_loss = self.loss_func(logits.squeeze(), labels.squeeze())
        score = self.val_metric(pred_proba, labels.squeeze())
        self.log(
            "valid_score_step",
            score,
            on_step=True,
            on_epoch=False,
            prog_bar=False,
            add_dataloader_idx=True,
            logger=True,
        )
        self.log(
            "valid_loss_step",
            valid_loss,
            on_step=True,
            on_epoch=False,
            prog_bar=False,
            add_dataloader_idx=True,
            logger=True,
        )
        val_logs = {
            "valid_loss": valid_loss,
            "valid_preds_step": preds,
            "valid_score_step": score,
            "valid_pred_proba_step": pred_proba,
            "valid_labels_step": labels.squeeze().cpu(),
        }
        self.val_step_outputs.append(val_logs)
        return valid_loss

    def evaluate_epoch_end(self, outputs, stage):
        """Custom method to be called to evaluate performance on epoch end"""
        loss_avg = torch.stack([x[f"{stage}_loss"] for x in outputs]).mean()  # floats
        # do other evaluations
        self.log(
            f"{stage}_loss",
            loss_avg,
            on_step=False,
            on_epoch=True,
            prog_bar=True,
            logger=True,
        )

    def on_training_epoch_end(self):
        # outputs from all training steps
        self.evaluate_epoch_end(self.train_step_outputs, stage="train")
        self.log(
            "train_score",
            self.train_metric.compute(),
            on_step=False,
            on_epoch=True,
            prog_bar=True,
        )
        self.train_metric.reset()  # reset after epoch
        self.train_step_outputs.clear()  # free memory

    def on_validation_epoch_end(self):
        # outputs from all training steps
        self.evaluate_epoch_end(self.val_step_outputs, stage="valid")
        self.log(
            "valid_score",
            self.val_metric.compute(),
            on_step=False,
            on_epoch=True,
            prog_bar=True,
        )
        self.val_metric.reset()  # reset after epoch
        self.val_step_outputs.clear()  # free memory

    def configure_optimizers(self):
        params = self.model.parameters()
        optimizer = torch.optim.Adam(params=params, lr=self.lr, weight_decay=self.wd)
        lr_scheduler = None
        """
        {
            "scheduler": torch.optim.lr_scheduler.ReduceLROnPlateau(
                optimizer, patience=self.patience, verbose=self.verbose
            ),
            "monitor": "valid_loss",
            "interval": "epoch",  # for ReduceLROnPlateau
        }
        """
        return (
            {"optimizer": optimizer, "lr_scheduler": lr_scheduler}
            if lr_scheduler is not None
            else optimizer
        )

In [ ]:
from captum_trial.model import ConvNet, ConvBnRelu
from captum_trial.data import get_dls

In [ ]:
dls = get_dls(batch_size=20) # has train and valid dataloaders

Files already downloaded and verified
Files already downloaded and verified


In [ ]:
pl_model = CIFARLightningModule(model=ConvNet, dls=dls, num_classes=10)
callbacks = []
trainer = pl.Trainer(max_epochs=5, callbacks=callbacks, num_sanity_val_steps=1)

/mnt/data/captum_trial/.venv/lib/python3.11/site-packages/torch/nn/modules/lazy.py:180: UserWarning: Lazy modules are a new feature under heavy development so changes to the API or functionality can happen at any moment.
  warnings.warn('Lazy modules are a new feature under heavy development '
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs


In [ ]:
trainer.fit(model=pl_model)

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name         | Type             | Params
--------------------------------------------------
0 | model        | ConvNet          | 5.2 K 
1 | loss_func    | CrossEntropyLoss | 0     
2 | train_metric | MulticlassAUROC  | 0     
3 | val_metric   | MulticlassAUROC  | 0     
--------------------------------------------------
5.2 K     Trainable params
0         Non-trainable params
5.2 K     Total params
0.021     Total estimated model params size (MB)


/mnt/data/captum_trial/.venv/lib/python3.11/site-packages/torchmetrics/utilities/prints.py:42: UserWarning: No positive samples in targets, true positive value should be meaningless. Returning zero tensor in true positive score
  warnings.warn(*args, **kwargs)  # noqa: B028


Epoch 4: 100%|██████████| 2500/2500 [00:57<00:00, 43.52it/s, v_num=37, valid_loss=1.490, valid_score=0.909]

`Trainer.fit` stopped: `max_epochs=5` reached.


Epoch 4: 100%|██████████| 2500/2500 [01:01<00:00, 40.56it/s, v_num=37, valid_loss=1.490, valid_score=0.909]


# Visualize the filters

In [ ]:
trained_model = trainer.model.model

In [ ]:
conv_layers = [x.conv for x in trained_model.children() if isinstance(x, ConvBnRelu) if hasattr(x, 'conv')]
conv_layers

[Conv2d(3, 16, kernel_size=(3, 3), stride=(1, 1)),
 Conv2d(16, 32, kernel_size=(3, 3), stride=(1, 1))]

In [ ]:
list(conv_layers[0].conv.parameters())[0].shape

torch.Size([16, 3, 3, 3])

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()